# Symmetry reduction

**Adapted from**: [SymbolicWedderburn example](https://github.com/kalmarek/SymbolicWedderburn.jl/blob/tw/ex_sos/examples/ex_C4.jl)

In [1]:
import MutableArithmetics
const MA = MutableArithmetics
using MultivariatePolynomials
const MP = MultivariatePolynomials
using MultivariateBases
const MB = MultivariateBases

using DynamicPolynomials
@polyvar x[1:4]

(DynamicPolynomials.PolyVar{true}[x₁, x₂, x₃, x₄],)

We would like to find the minimum value of the polynomial

In [2]:
poly = sum(x) + sum(x.^2)

x₁² + x₂² + x₃² + x₄² + x₁ + x₂ + x₃ + x₄

As we can decouple the problem for each `x[i]` for which `x[i] + x[i]^2` has
minimum value 0.25, we would expect to get `-1` as answer.
Can this decoupling be exploited by SumOfSquares as well ?
For this, we need to use a certificate that can exploit the permutation symmetry of the polynomial.
Symmetry reduction is still a work in progress in SumOfSquares, so we include the following files that will be incorporated into SumOfSquares.jl once SymbolicWedderburn.jl is released:

In [3]:
using SumOfSquares
include(joinpath(dirname(dirname(pathof(SumOfSquares))), "examples", "symmetry.jl"))

    Updating git-repo `https://github.com/kalmarek/SymbolicWedderburn.jl`
   Resolving package versions...
  No Changes to `~/work/SumOfSquares.jl/SumOfSquares.jl/docs/Project.toml`
  No Changes to `~/work/SumOfSquares.jl/SumOfSquares.jl/docs/Manifest.toml`


isblockdim (generic function with 2 methods)

We define the symmetry group as a permutation group in the variables.
In order to do that, we define the action of a permutation on a monomial
as the monomial obtained after permuting the variables.

In [4]:
using PermutationGroups
function action(mono::AbstractMonomial, p::Perm)
    v = variables(mono)
    MP.substitute(MP.Eval(), mono, v => [v[i^p] for i in eachindex(v)])
end
function action(term::MP.AbstractTerm, el::Perm)
    return MP.coefficient(term) * action(MP.monomial(term), el)
end
function action(poly::MP.AbstractPolynomial, el::Perm)
    return MP.polynomial([action(term, el) for term in MP.terms(poly)])
end

G = PermGroup([perm"(1,2,3,4)"])

Permutation group on 1 generator generated by
 (1,2,3,4)

We can use this certificate as follows:

In [5]:
import CSDP
solver = CSDP.Optimizer
model = Model(solver)
@variable(model, t)
@objective(model, Max, t)
certificate = SymmetricIdeal(Certificate.MaxDegree(SOSCone(), MonomialBasis, maxdegree(poly)), G, action)
con_ref = @constraint(model, poly - t in SOSCone(), ideal_certificate = certificate)
optimize!(model)
value(t)

Iter:  7 Ap: 9.00e-01 Pobj:  0.0000000e+00 Ad: 1.00e+00 Dobj:  5.8619776e-14 
Success: SDP solved
Primal objective value: 0.0000000e+00 
Dual objective value: 0.0000000e+00 
Relative primal infeasibility: 1.75e-09 
Relative dual infeasibility: 2.83e-50 
Real Relative Gap: 0.00e+00 
XZ Relative Gap: 8.00e-50 
DIMACS error measures: 2.63e-09 0.00e+00 2.83e-50 0.00e+00 0.00e+00 8.00e-50
CSDP 6.2.0
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
Iter:  1 Ap: 9.92e-01 Pobj: -4.6205388e+00 Ad: 9.53e-01 Dobj:  8.8960402e+00 
Iter:  2 Ap: 1.00e+00 Pobj: -2.3232300e+00 Ad: 9.15e-01 Dobj:  2.4150779e+00 
Iter:  3 Ap: 1.00e+00 Pobj: -1.0496347e+00 Ad: 8.98e-01 Dobj: -3.3505944e-01 
Iter:  4 Ap: 1.00e+00 Pobj: -1.0058358e+00 Ad: 9.36e-01 Dobj: -9.4074669e-01 
Iter:  5 Ap: 1.00e+00 Pobj: -1.0004335e+00 Ad: 9.56e-01 Dobj: -9.9613005e-01 
Iter:  6 Ap: 1.00e+00 Pobj: -1.0000297e+00 Ad: 9.88e-01 Dobj: -9.9986684e-01 
Iter:  7 Ap: 1.00e+00 Pobj: -1.0000017e+00 Ad: 1.00e+00 

-1.0000000002081693

We indeed find `-1`, let's verify that symmetry was exploited:

In [6]:
for g in gram_matrix(con_ref).sub_gram_matrices
    println(g.basis.polynomials)
end

DynamicPolynomials.Polynomial{true, Float64}[x₁ + x₂ + x₃ + x₄, 1.0]
DynamicPolynomials.Polynomial{true, Float64}[x₁ - x₃]
DynamicPolynomials.Polynomial{true, Float64}[x₂ - x₄]
DynamicPolynomials.Polynomial{true, Float64}[x₁ - x₂ + x₃ - x₄]


---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*